## Linear classifier benchmarking

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import hyperdt.benchmarking as benchmarking

from hyperdt.product_space_DT import ProductSpace
from hyperdt.product_space_perceptron import mix_curv_perceptron
from hyperdt.product_space_svm import mix_curv_svm

### Product space perceptron and SVM 

In [3]:
# Generate data
sig = [(5, -1), (5, 0), (5, .5)]
ps = ProductSpace(sig, seed=0)
ps.sample_clusters(1000, 4, cov_scale=0.3)
ps.split_data()

In [4]:
mix_component = benchmarking.sig_to_mix_component(sig)
embed_data = benchmarking.make_embed_data(ps.X, ps.X_train, ps.X_test, ps.y_train, ps.y_test, sig)

In [7]:
# Fit product space perceptron
ps_perc = mix_curv_perceptron(mix_component, embed_data, multiclass=True, max_round=100, max_update=1000)
ps_perc_score = ps_perc.process_data()

 800 samples finished. 191
 1 rounds finished.
 800 samples finished. 402
 2 rounds finished.
 800 samples finished. 627
 3 rounds finished.
 800 samples finished. 857
 4 rounds finished.
 800 samples finished. 1085
 5 rounds finished.
 800 samples finished. 1312
 6 rounds finished.
 800 samples finished. 1540
 7 rounds finished.
 800 samples finished. 1766
 8 rounds finished.
 800 samples finished. 1997
 9 rounds finished.
 800 samples finished. 2229
 10 rounds finished.
 800 samples finished. 2459
 11 rounds finished.
 800 samples finished. 2690
 12 rounds finished.
 800 samples finished. 2922
 13 rounds finished.
 800 samples finished. 3156
 14 rounds finished.
 800 samples finished. 3388
 15 rounds finished.
 800 samples finished. 3620
 16 rounds finished.
 800 samples finished. 3850
 17 rounds finished.
 800 samples finished. 4080
 18 rounds finished.
 800 samples finished. 4311
 19 rounds finished.
 800 samples finished. 4543
 20 rounds finished.
 800 samples finished. 4775
 21 r

In [10]:
# Fit product space SVM
ps_svm = mix_curv_svm(mix_component, embed_data)
ps_svm_score = ps_svm.process_data()

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (8001 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        

### Benchmarking for HyperDT/RF, sklearn DT/RF, perceptron and SVM

In [3]:
NUM_POINTS = 1000
NUM_CLASSES = 4
signatures = [
    [(5, -1), (5, -1)],
    [(5, 1), (5, 1)],
    [(5, -1), (5, 1)],
    [(2, -1), (2, -1), (2, -1), (2, -1), (2, -1)],
    [(2, 1), (2, 1), (2, 1), (2, 1), (2, 1)],
    [(2, -1), (2, -1), (2, 0), (2, 1), (2, 1)]
]

In [4]:
seed = 0
results = []

scores_tuple = benchmarking.compute_scores_by_signature(signatures, NUM_POINTS, NUM_CLASSES, seed=seed,
                                                        max_depth=3, n_seeds=10, metric="f1")
rnd_seeds = scores_tuple[0]
psdt_scores_by_signature = scores_tuple[1]
psrf_scores_by_signature = scores_tuple[2]
dt_scores_by_signature = scores_tuple[3]
rf_scores_by_signature = scores_tuple[4]
for signature, psdt_score, psrf_score, dt_score, rf_score in zip(signatures, psdt_scores_by_signature,
                                                                       psrf_scores_by_signature, dt_scores_by_signature,
                                                                       rf_scores_by_signature):
    results.append({
        "signature": signature,
        "psdt_score": psdt_score,
        "psrf_score": psrf_score,
        "dt_score": dt_score,
        "rf_score": rf_score
    })

results = pd.DataFrame(results)
results

100%|██████████| 60/60 [07:43<00:00,  7.73s/it]


,signature,psdt_score,psrf_score,dt_score,rf_score
0,"[(5, -1), (5, -1)]","[0.9551573426573426, 0.9785456187895212, 0.926...","[0.9588899598884433, 0.9785456187895212, 0.925...","[0.9260686177724762, 0.8727146254515196, 0.882...","[0.9417164357381749, 0.9015478591768107, 0.882..."
1,"[(5, 1), (5, 1)]","[0.5155353876100239, 0.44970776424264797, 0.56...","[0.4592301855278525, 0.48272736223796936, 0.56...","[0.48822628936764145, 0.4580204903141638, 0.49...","[0.481566390413662, 0.5337546990988425, 0.5571..."
2,"[(5, -1), (5, 1)]","[0.9551573426573426, 0.9785456187895212, 0.926...","[0.9588899598884433, 0.9785456187895212, 0.925...","[0.9260686177724762, 0.8679546771019484, 0.882...","[0.7804181816709451, 0.9345250105691996, 0.924..."
3,"[(2, -1), (2, -1), (2, -1), (2, -1), (2, -1)]","[0.5221285999974525, 0.9486712901332646, 0.702...","[0.7866836859483919, 0.9434230673361108, 0.704...","[0.7072916666666667, 0.9232354034144317, 0.685...","[0.7223639642293134, 0.9284646061814557, 0.696..."
4,"[(2, 1), (2, 1), (2, 1), (2, 1), (2, 1)]","[0.46734234512476247, 0.7073675287501551, 0.38...","[0.6250351681035349, 0.6950405173757207, 0.400...","[0.3502872814870276, 0.6943961257927083, 0.395...","[0.5095859808705673, 0.7370441503040153, 0.392..."
5,"[(2, -1), (2, -1), (2, 0), (2, 1), (2, 1)]","[0.7492963230343687, 0.9486712901332646, 0.714...","[0.7910290460131393, 0.9434230673361108, 0.685...","[0.7005743577075099, 0.9415738498789346, 0.709...","[0.7408216933542249, 0.9350568255056898, 0.688..."


In [5]:
for signature, (i, row) in zip(signatures, results.iterrows()):
    psdt_scores = row["psdt_score"]
    psrf_scores = row["psrf_score"]
    dt_scores = row["dt_score"]
    rf_scores = row["rf_score"]
    s = []
    for component in signature:
        if component[1] < 0:
            s.append("H(" + f"{component[0]}, {-component[1]}" + ")")
        elif component[1] == 0:
            s.append("E(" + f"{component[0]}" + ")")
        else:
            s.append("S(" + f"{component[0]}, {component[1]}" + ")")
    print(" x ".join(s))
    print(f"Avg PSDT Score: {np.mean(psdt_scores) * 100:.1f} +/- {np.std(psdt_scores) / np.sqrt(len(psdt_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg DT Score: {np.mean(dt_scores) * 100:.1f} +/- {np.std(dt_scores) / np.sqrt(len(dt_scores)) * 1.96 * 100 :.1f}")
    
    t, p = stats.ttest_rel(psdt_scores, dt_scores)
    if p < .05:
        print("PSDT and DT are significantly different")
    
    print(f"Avg PSRF Score: {np.mean(psrf_scores) * 100:.1f} +/- {np.std(psrf_scores) / np.sqrt(len(psrf_scores)) * 1.96 * 100 :.1f}", end=", ")
    print(f"Avg RF Score: {np.mean(rf_scores) * 100:.1f} +/- {np.std(rf_scores) / np.sqrt(len(rf_scores)) * 1.96 * 100 :.1f}")

    t, p = stats.ttest_rel(psrf_scores, rf_scores)
    if p < .05:
        print("PSRF and RF are significantly different")
    print("")

H(5, 1) x H(5, 1)
Avg PSDT Score: 95.0 +/- 2.8, Avg DT Score: 88.6 +/- 4.3
PSDT and DT are significantly different
Avg PSRF Score: 95.3 +/- 2.9, Avg RF Score: 90.0 +/- 3.6
PSRF and RF are significantly different

S(5, 1) x S(5, 1)
Avg PSDT Score: 51.4 +/- 3.3, Avg DT Score: 47.9 +/- 2.4
Avg PSRF Score: 52.4 +/- 4.0, Avg RF Score: 50.1 +/- 2.2

H(5, 1) x S(5, 1)
Avg PSDT Score: 95.0 +/- 2.8, Avg DT Score: 88.8 +/- 4.4
PSDT and DT are significantly different
Avg PSRF Score: 95.3 +/- 2.9, Avg RF Score: 87.5 +/- 3.8
PSRF and RF are significantly different

H(2, 1) x H(2, 1) x H(2, 1) x H(2, 1) x H(2, 1)
Avg PSDT Score: 67.6 +/- 9.0, Avg DT Score: 67.5 +/- 7.6
Avg PSRF Score: 72.0 +/- 8.0, Avg RF Score: 68.6 +/- 7.0

S(2, 1) x S(2, 1) x S(2, 1) x S(2, 1) x S(2, 1)
Avg PSDT Score: 44.4 +/- 6.9, Avg DT Score: 42.7 +/- 6.8
Avg PSRF Score: 47.8 +/- 7.1, Avg RF Score: 47.0 +/- 6.9

H(2, 1) x H(2, 1) x E(2) x S(2, 1) x S(2, 1)
Avg PSDT Score: 71.0 +/- 7.6, Avg DT Score: 68.4 +/- 7.7
PSDT and DT a